# Start Section

In [4]:
##import important library
import io
import pandas as pd
import requests

In [139]:
##reading parquet file
df = pd.read_csv('yellow_tripdata_2023-11(cut)1.csv')

In [140]:
##remove duplicate data
df = df.drop_duplicates().reset_index(drop=True)

In [141]:
duplicates = df.duplicated()

print(df[duplicates]) ##showing duplicate data only

##count the number of duplicate data
number_of_duplicates = duplicates.sum()
print("Number of duplicate rows:", number_of_duplicates)


Empty DataFrame
Columns: [VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, Airport_fee]
Index: []
Number of duplicate rows: 0


In [144]:
df.head()

,trip_index_id,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,0,1,11/1/2023 0:03,11/1/2023 1:04,2,13.60,1,N,132,26,2,61.8,2.75,0.5,0.00,0.00,1.0,66.05,0.0,1.75
1,1,1,11/1/2023 0:03,11/1/2023 0:23,0,3.50,1,N,140,7,1,20.5,3.50,0.5,5.10,0.00,1.0,30.60,2.5,0.00
2,2,2,10/31/2023 23:58,11/1/2023 0:54,4,18.61,2,N,132,230,1,70.0,0.00,0.5,16.54,6.94,1.0,99.23,2.5,1.75
3,3,2,11/1/2023 0:03,11/1/2023 0:04,1,0.39,1,N,236,236,1,4.4,1.00,0.5,1.88,0.00,1.0,11.28,2.5,0.00
4,4,2,11/1/2023 0:06,11/1/2023 0:14,1,1.20,1,N,236,141,1,10.0,1.00,0.5,3.00,0.00,1.0,18.00,2.5,0.00


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3339713 entries, 0 to 3339712
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   trip_id                int64         
 1   VendorID               int32         
 2   tpep_pickup_datetime   datetime64[ns]
 3   tpep_dropoff_datetime  datetime64[ns]
 4   passenger_count        float64       
 5   trip_distance          float64       
 6   RatecodeID             float64       
 7   store_and_fwd_flag     object        
 8   PULocationID           int32         
 9   DOLocationID           int32         
 10  payment_type           int64         
 11  fare_amount            float64       
 12  extra                  float64       
 13  mta_tax                float64       
 14  tip_amount             float64       
 15  tolls_amount           float64       
 16  improvement_surcharge  float64       
 17  total_amount           float64       
 18  congestion_surcharge  

In [143]:
# ##Insert some forengin key for referencing in dim table
# df.insert(0, 'trip_id', ['r' + str(i) for i in range(1, len(df) + 1)]) #generate trip_id
# df.insert(2, 'datetime_id', 1000 + (df.index * 3)) #generate datetime_id
# df.insert(8, 'location_id', ['l' + str(i) for i in range(1, len (df) +1)]) #generate location_id
df.insert(0, 'trip_index_id', df.index)

In [90]:
##Drop incase it has to change or fix on the key 
df.drop('trip_id', axis=1, inplace=True)
# df.drop ('datetime_id', axis=1, inplace=True)
# df.drop ('location_id', axis = 1 , inplace =True)

# Dim_Datetime Section

In [145]:
# Create Dim_Datetime table
dim_datetime = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)

# Generate a primary key for this dim
dim_datetime['datetimeindex_id'] = dim_datetime.index

# Extract datetime components for pickup
dim_datetime['pick_datetime'] = pd.to_datetime(dim_datetime['tpep_pickup_datetime'])
dim_datetime['pick_hour'] = pd.to_datetime(dim_datetime['tpep_pickup_datetime']).dt.hour
dim_datetime['pick_day'] = pd.to_datetime(dim_datetime['tpep_pickup_datetime']).dt.day
dim_datetime['pick_month'] = pd.to_datetime(dim_datetime['tpep_pickup_datetime']).dt.month
dim_datetime['pick_year'] = pd.to_datetime(dim_datetime['tpep_pickup_datetime']).dt.year
dim_datetime['pick_weekday'] = pd.to_datetime(dim_datetime['tpep_pickup_datetime']).dt.weekday

# Extract datetime components for dropoff
dim_datetime['drop_datetime']= pd.to_datetime(dim_datetime['tpep_dropoff_datetime'])
dim_datetime['drop_hour'] = pd.to_datetime(dim_datetime['tpep_dropoff_datetime']).dt.hour
dim_datetime['drop_day'] = pd.to_datetime(dim_datetime['tpep_dropoff_datetime']).dt.day
dim_datetime['drop_month'] = pd.to_datetime(dim_datetime['tpep_dropoff_datetime']).dt.month
dim_datetime['drop_year'] = pd.to_datetime(dim_datetime['tpep_dropoff_datetime']).dt.year
dim_datetime['drop_weekday'] = pd.to_datetime(dim_datetime['tpep_dropoff_datetime']).dt.weekday

# Reorder columns as needed and display the head of the DataFrame
dim_datetime = dim_datetime[['datetimeindex_id', 'pick_datetime', 'pick_hour', 'pick_day', 
                             'pick_month', 'pick_year', 'pick_weekday',
                             'drop_datetime', 'drop_hour', 'drop_day', 
                             'drop_month', 'drop_year', 'drop_weekday']]

In [146]:
##Checking Dim Date_Time
dim_datetime

,datetimeindex_id,pick_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,drop_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2023-11-01 00:03:00,0,1,11,2023,2,2023-11-01 01:04:00,1,1,11,2023,2
1,1,2023-11-01 00:03:00,0,1,11,2023,2,2023-11-01 00:23:00,0,1,11,2023,2
2,2,2023-10-31 23:58:00,23,31,10,2023,1,2023-11-01 00:54:00,0,1,11,2023,2
3,3,2023-11-01 00:03:00,0,1,11,2023,2,2023-11-01 00:04:00,0,1,11,2023,2
4,4,2023-11-01 00:06:00,0,1,11,2023,2,2023-11-01 00:14:00,0,1,11,2023,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99990,99990,2023-11-01 20:12:00,20,1,11,2023,2,2023-11-01 20:30:00,20,1,11,2023,2
99991,99991,2023-11-01 20:47:00,20,1,11,2023,2,2023-11-01 21:03:00,21,1,11,2023,2
99992,99992,2023-11-01 20:25:00,20,1,11,2023,2,2023-11-01 20:47:00,20,1,11,2023,2
99993,99993,2023-11-01 20:27:00,20,1,11,2023,2,2023-11-01 21:19:00,21,1,11,2023,2


   # Dim_Ratecode Section

In [148]:
rate_code_type = {
    1: 'Standard Rate',
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated fare",
    6: "Group Ride"
}

dim_ratecode = df[['RatecodeID']].reset_index(drop=True)
dim_ratecode['ratecodeindex_id'] = dim_ratecode.index
dim_ratecode['ratecode_id'] = dim_ratecode ['RatecodeID']
dim_ratecode['ratecode_name'] = dim_ratecode ['RatecodeID'].map(rate_code_type)
dim_ratecode = dim_ratecode [['ratecodeindex_id','ratecode_id','ratecode_name']]


In [149]:
dim_ratecode 

,ratecodeindex_id,ratecode_id,ratecode_name
0,0,1,Standard Rate
1,1,1,Standard Rate
2,2,2,JFK
3,3,1,Standard Rate
4,4,1,Standard Rate
...,...,...,...
99990,99990,1,Standard Rate
99991,99991,1,Standard Rate
99992,99992,1,Standard Rate
99993,99993,2,JFK


# Dim_Payment_type Section

In [150]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
dim_payment_type = df[['payment_type']].reset_index(drop=True)
dim_payment_type['payment_typeindex_id'] = dim_payment_type.index
dim_payment_type['payment_type_id'] = dim_payment_type ['payment_type']
dim_payment_type['payment_type_name'] = dim_payment_type['payment_type'].map(payment_type_name)
dim_payment_type = dim_payment_type[['payment_typeindex_id','payment_type_id','payment_type_name']]

In [151]:
dim_payment_type.head(100)

,payment_typeindex_id,payment_type_id,payment_type_name
0,0,2,Cash
1,1,1,Credit card
2,2,1,Credit card
3,3,1,Credit card
4,4,1,Credit card
...,...,...,...
95,95,2,Cash
96,96,1,Credit card
97,97,1,Credit card
98,98,2,Cash


# Dim_Store_and_FWD_Flag Section

In [152]:
store_and_fwd_flag = {
    'Y': 'store and forward trip',
    'N': 'Not a store and forward trip'
}

dim_store_and_fwd_flag = df [['store_and_fwd_flag']].reset_index(drop=True)
dim_store_and_fwd_flag ['store_and_fwd_flag_index_id'] = dim_store_and_fwd_flag.index
dim_store_and_fwd_flag ['store_and_fwd_flag_id'] = dim_store_and_fwd_flag ['store_and_fwd_flag']
dim_store_and_fwd_flag ['store_and_fwd_flag_type'] = dim_store_and_fwd_flag['store_and_fwd_flag'].map (store_and_fwd_flag)
dim_store_and_fwd_flag = dim_store_and_fwd_flag [['store_and_fwd_flag_index_id','store_and_fwd_flag_id','store_and_fwd_flag_type']]

In [153]:
dim_store_and_fwd_flag.head()

,store_and_fwd_flag_index_id,store_and_fwd_flag_id,store_and_fwd_flag_type
0,0,N,Not a store and forward trip
1,1,N,Not a store and forward trip
2,2,N,Not a store and forward trip
3,3,N,Not a store and forward trip
4,4,N,Not a store and forward trip


# Dim_Vendor Section

In [154]:
vendor_type_list = {
    1: 'Creative Mobile Technologies,LLC',
    2: 'Verifone Inc'
}

dim_vendor = df [['VendorID']].reset_index(drop=True)
dim_vendor ['vendorindex_id'] = dim_vendor.index
dim_vendor ['vendor_type'] = dim_vendor ['VendorID'].map(vendor_type_list)
dim_vendor ['vendor_id']  = dim_vendor ['VendorID']
dim_vendor = dim_vendor [['vendorindex_id','vendor_id','vendor_type']]

In [155]:
dim_vendor.head()

,vendorindex_id,vendor_id,vendor_type
0,0,1,"Creative Mobile Technologies,LLC"
1,1,1,"Creative Mobile Technologies,LLC"
2,2,2,Verifone Inc
3,3,2,Verifone Inc
4,4,2,Verifone Inc


# Dim_Location Section

In [156]:
dim_location = df[['PULocationID', 'DOLocationID']].reset_index(drop=True)
dim_location['locationindex_id'] = dim_location.index
dim_location['PUlocationCode'] = dim_location['PULocationID'] 
dim_location['DOlocationCode'] = dim_location['DOLocationID']
dim_location = dim_location[['locationindex_id', 'PUlocationCode', 'DOlocationCode']]


In [113]:
dim_location.head()

,locationindex_id,PUlocationCode,DOlocationCode
0,0,132,26
1,1,140,7
2,2,132,230
3,3,236,236
4,4,236,141


# Fact_Trip Section

In [157]:
fact_trip= pd.DataFrame()
fact_trip['trip_id'] = df.index

##merge table
fact_trip = fact_trip.merge (dim_ratecode,left_on='trip_id',right_on='ratecodeindex_id')\
                     .merge (dim_payment_type,left_on='trip_id',right_on='payment_typeindex_id')\
                     .merge (dim_store_and_fwd_flag,left_on='trip_id',right_on='store_and_fwd_flag_index_id')\
                     .merge (dim_vendor,left_on='trip_id',right_on='vendorindex_id')\
                     .merge (dim_datetime,left_on='trip_id',right_on='datetimeindex_id')\
                     .merge (dim_location,left_on='trip_id',right_on='locationindex_id')\
                     .merge (df, left_on='trip_id', right_on='trip_index_id')

fact_trip = fact_trip [[
                        
                        'trip_id','vendorindex_id','datetimeindex_id','locationindex_id','ratecodeindex_id',
                        'payment_typeindex_id','store_and_fwd_flag_id','passenger_count','trip_distance',
                        'fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge',
                        'congestion_surcharge','Airport_fee','total_amount'
                       
                      ]]

In [160]:
fact_trip

,trip_id,vendorindex_id,datetimeindex_id,locationindex_id,ratecodeindex_id,payment_typeindex_id,store_and_fwd_flag_id,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee,total_amount
0,0,0,0,0,0,0,N,2,13.60,61.8,2.75,0.5,0.00,0.00,1.0,0.0,1.75,66.05
1,1,1,1,1,1,1,N,0,3.50,20.5,3.50,0.5,5.10,0.00,1.0,2.5,0.00,30.60
2,2,2,2,2,2,2,N,4,18.61,70.0,0.00,0.5,16.54,6.94,1.0,2.5,1.75,99.23
3,3,3,3,3,3,3,N,1,0.39,4.4,1.00,0.5,1.88,0.00,1.0,2.5,0.00,11.28
4,4,4,4,4,4,4,N,1,1.20,10.0,1.00,0.5,3.00,0.00,1.0,2.5,0.00,18.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99990,99990,99990,99990,99990,99990,99990,N,1,2.30,15.6,3.50,0.5,2.00,0.00,1.0,2.5,0.00,22.60
99991,99991,99991,99991,99991,99991,99991,N,1,1.80,12.8,3.50,0.5,3.55,0.00,1.0,2.5,0.00,21.35
99992,99992,99992,99992,99992,99992,99992,N,1,7.49,33.1,1.00,0.5,7.62,0.00,1.0,2.5,0.00,45.72
99993,99993,99993,99993,99993,99993,99993,N,2,21.35,70.0,0.00,0.5,0.00,0.00,1.0,2.5,1.75,75.75
